### **N-gram Language model**
[ 19EC10086 Shruti Shreyasi ]

Run on Google Colaboratory

Necessary imports

In [1]:
from cmath import exp, log

Language Model Class Definition

In [2]:
class NGramModel:
    def __init__(self, N, corpus, test_corpus):
        '''
            initialization of the class
            args
                N :: is the number corresponding to the N gram
                corpus :: is the train dataset
                test_corpus :: is the test dataset
        '''
        self.N = N
        self.corpus = corpus
        self.test_corpus = test_corpus

    def preprocess_data(self):
        '''
            i) add start and end tokens to the dataset:
                    unigrams: start and end tokens are not added because they have no effect
                    other n-grams: add N-1 start tokens and 1 end token
            ii) words occuring only once are treated as UNK
        '''
        self.count_test_corpus = 0              # keep track of test corpus size

        # i) add start and end tokens to the dataset
        add_start_token = ''                    # keep track of start tokens
        for i in range(self.N-1):
            add_start_token += '<START> '
        add_end_token = ' <END>'                # keep track of end tokens
        if self.N == 1:
            add_end_token = ''
        # add start and end tokens
        for i in range(len(self.corpus)):
            self.corpus[i] = add_start_token + self.corpus[i] + add_end_token
        for i in range(len(self.test_corpus)):
            self.test_corpus[i] = add_start_token + self.test_corpus[i] + add_end_token

        # ii) words occuring only once are treated as UNK
        for sentence in self.test_corpus:
            self.count_test_corpus += len(sentence.split())
        vocabulary_before_seeding = {}         # keep track of tokens occuring only once
        for sentence in self.corpus:
            for word in sentence.split():
                if word in vocabulary_before_seeding:
                    vocabulary_before_seeding[word] += 1
                else:
                    vocabulary_before_seeding[word] = 1
        # remove words with freq 1 from train corpus
        for i in range(len(self.corpus)):
            sentence = self.corpus[i]
            sent = ''
            for word in sentence.split():
                if vocabulary_before_seeding[word] <= 1:
                    sent += '<UNK> '
                else:
                    sent += word + ' '
            sent = sent[:len(sent)-1]
            self.corpus[i] = sent
        # remove words with freq 1/0 in train set from test corpus
        for i in range(len(self.test_corpus)):
            sentence = self.test_corpus[i]
            sent = ''
            for word in sentence.split():
                if word not in vocabulary_before_seeding or vocabulary_before_seeding[word] <= 1:
                    sent += '<UNK> '
                else:
                    sent += word + ' '
            sent = sent[:len(sent)-1]
            self.test_corpus[i] = sent
            
    def n_gram_model(self):
        '''
            model definition having n gram and n-1 gram
            i) store frequencies of all n-grams and n-1 grams
        '''
        self.preprocess_data()
        self.count_total_unique_n_grams = 0            # keep track of total number of unique n grams
        self.count_total_unique_less_grams = 0         # keep track of total number of unique n-1 grams
        self.n_gram_count = {}                         # frequency mapping for n grams
        self.count_total_non_unique_less_grams = 0     # keep track of total number of n grams
        self.count_total_non_unique_n_grams = 0        # keep track of total number of n-1 grams
        self.less_gram_count = {}                      # frequency mapping for n-1 grams
        self.unique_words = {}                         # frequency mapping for unique tokens
        self.unique_words_count = 0                    # keep track of total number of unique tokens

        for sentence in self.corpus:
            str = sentence.split()
            # n grams handling
            for i in range(len(str)-self.N+1):
                self.count_total_non_unique_n_grams += 1
                n_gram = str[i:i+self.N]
                n = '' # store all n grams as string
                for word in n_gram:
                    n = n + word + ' '
                n = n[:len(n)-1]                      
                if n in self.n_gram_count:
                    self.n_gram_count[n] += 1
                else:
                    self.n_gram_count[n] = 1
                    self.count_total_unique_n_grams += 1
                if str[i+self.N-1] not in self.unique_words:
                    self.unique_words[str[i+self.N-1]] = 1
                    self.unique_words_count += 1

            # n-1 grams handling
            for i in range(len(str)-self.N+1):
                self.count_total_non_unique_less_grams += 1
                n_gram = str[i:i+self.N-1]
                n = '' # store all n-1 grams as string
                for word in n_gram:
                    n = n + word + ' '
                n = n[:len(n)-1]
                if n in self.less_gram_count:
                    self.less_gram_count[n] += 1
                else:
                    self.less_gram_count[n] = 1
                    self.count_total_unique_less_grams += 1

    def perplexity(self, sentence):
        '''
            function to calculate perplexity of a sentence
            i) laplace smoothing is applied for n-grams
            ii) for unigram no smoothing is required 
                if word is not found it is treated as UNK
            args:
                sentence:: sentence for which perplexity(PP) has to be computed
        '''
        PP = 0
        str = sentence.split()
        if self.N == 1:
            for i in range (len(str)):
                # probability = count of word / vocabulary size (+1 to accomodate UNK)
                PP = PP + log(self.n_gram_count.get(str[i],1)) - log(self.count_total_non_unique_n_grams+1)
            # return perplexity
            return (exp((-PP)/len(str))).real
        for i in range(len(str)-self.N+1):
            # determine n-1 gram 
            m_gram = str[i:i+self.N-1]
            m = ''
            for word in m_gram:
                m = m + word + ' '
            m = m[:len(m)-1]
            # determine n gram
            n = m + ' ' + str[i+self.N-1]
            # probability = count of ngram + 1 / count of n-1gram + V (laplace smoothing)
            PP = PP + log(self.n_gram_count.get(n,0)+1) - log(self.less_gram_count.get(m,0) + (self.unique_words_count))
        # return perplexity
        return (exp((-PP)/len(str))).real

    def get_perplexity(self):
        '''
            function to calculate perplexity of test corpus
            perplexity of a corpus = (product of probabilities of each sentence)^(1/corpus size)
        '''
        self.n_gram_model()
        self.length_test_set = 0                         # keep track of test set size
        for sentence in self.test_corpus:
            self.length_test_set += len(sentence.split())
        PP = 0
        for sentence in self.test_corpus:
            # convert perplexity back to log probability
            PP += log(self.perplexity(sentence))*(len(sentence.split())-self.N+1-1)
        PP = exp(PP/self.count_test_corpus)
        # return perplexity
        return (PP.real)

    def generate_sentence(self, seed = '', max_length = 25):
        '''
            function to generate sentence based on the maximum probable n-gram
            args:
                seed:: the partial sequence
                max_length:: maximum sequence length desirable
        '''
        self.n_gram_model() # get model parameters
        # add start tokens in case words fall short for generation
        m = len(seed.split())
        sentence = seed
        # add N-1 start tokens
        sentence = ('<START> ' * (self.N - 1)) + seed
        sentence = sentence.split()
        # keep generating words until END token or length limit is reached
        while sentence[len(sentence)-1] != '<END>' and (len(sentence)- self.N + 1 + m) < max_length:
            # extract the n-1 gram
            m_gram = ''
            for i in range(len(sentence)-self.N+1,len(sentence)):
                m_gram += sentence[i] + ' '
            m_gram = m_gram[:len(m_gram)-1]
            # determine the best candidate for next word based on minimum log PP score
            PP_minima = -1e100
            best_candidate = ''
            for word in self.unique_words:
                    n_gram = m_gram + ' ' + word
                    # minimize PP score
                    PP = log(self.n_gram_count.get(n_gram,0)+1) - log(self.less_gram_count.get(m_gram,0) + self.unique_words_count)
                    PP = PP.real
                    if PP > PP_minima:
                        best_candidate = word
                        PP_minima = PP
            sentence.append(best_candidate)
        if sentence[len(sentence)-1] != '<END>':
            sentence.append('<END>')
        # print the most probable sequence
        sentence = sentence[self.N - 1:len(sentence) - 1]
        print(' '.join([str(elem) for elem in sentence]))

train and test the model

In [3]:
# train the model and check its perplexity score on the test dataset
for i in range(1, 4):
    train = open('train.txt').read()
    test = open('test.txt').read()
    sentences = train.splitlines()
    test_sentences = test.splitlines()
    object = NGramModel(i, sentences, test_sentences)
    perplexity = object.get_perplexity()
    print('perplexity of {}-gram model is:: {}'.format(i, perplexity))
    del object

perplexity of 1-gram model is:: 777.7048283969436
perplexity of 2-gram model is:: 580.9471390295671
perplexity of 3-gram model is:: 1291.848782371178


test the generate sentence method for bigram onwards: 

(it can be noticed that the complexity of the sentences are more for greater values of N)

In [5]:
# generate the most likely sentence based on N-gram models
for i in range(2, 6):
    train = open('train.txt').read()
    test = open('test.txt').read()
    sentences = train.splitlines()
    test_sentences = test.splitlines()
    object = NGramModel(i, sentences, test_sentences)
    print('\nsentence generation by {}-gram model::'.format(i))
    object.generate_sentence('the sale', 20)
    del object


sentence generation by 2-gram model::
the sale of the company said

sentence generation by 3-gram model::
the sale of its common stock

sentence generation by 4-gram model::
the sale is part of a restructuring plan it expects pre tax operating loss of two cts a

sentence generation by 5-gram model::
the sale is part of a major program to divest several of its businesses representing about 200 mln
